# Test case: mobility
Contains all graphs, data sets and models for the mobility test case.

In [ ]:
from ipynb.fs.full.classes import *

# Variables and graphs
The use case consists of the following variables and granularities:
- Background characteristics (b): 0
- Vehicle count (c): 0
- Time (t): 0, 1, 2
- Infrastructure (i): 0, 1
- Origin destination matrix (od): 0
- Person (p): 0
- Modality (m): 0


In [ ]:
# create conversion graphs
# no conversion graph required in this case study
cg_b = ConversionGraph(variable_name = "b",
                         granularities = [0],
                       conversion_edges = [(0,0)])
cg_c = ConversionGraph(variable_name = "c",
                         granularities = [0],
                       conversion_edges = [(0,0)])
cg_t = ConversionGraph(variable_name = "t",
                         granularities = [0,1,2],
                       conversion_edges = [(0,0)])
cg_r = ConversionGraph(variable_name = "r",
                         granularities = [0,1],
                       conversion_edges = [(0,0)])
cg_od = ConversionGraph(variable_name = "od",
                         granularities = [0],
                       conversion_edges = [(0,0)])
cg_p = ConversionGraph(variable_name = "p",
                         granularities = [0],
                       conversion_edges = [(0,0)])
cg_m = ConversionGraph(variable_name = "m",
                         granularities = [0],
                       conversion_edges = [(0,0)])
# create aggregation graphs
ag_b = AggregationGraph(variable_name = "b",  
                         granularities = [0],
                         aggregation_edges = [(0,0)])
ag_c = AggregationGraph(variable_name = "c",  
                         granularities = [0],
                         aggregation_edges = [(0,0)])
ag_t = AggregationGraph(variable_name = "t",  # Variable Time
                         granularities = [0,1,2],
                         aggregation_edges = [(0,1), (1,2)])
ag_r = AggregationGraph(variable_name = "r",  # Variable Infrastructure
                         granularities = [0,1],
                         aggregation_edges = [(0,1), (1,0)])
ag_od = AggregationGraph(variable_name = "od",  
                         granularities = [0,1],
                         aggregation_edges = [(0,0)])
ag_p = AggregationGraph(variable_name = "p",  
                         granularities = [0],
                         aggregation_edges = [(0,0)])
ag_m = AggregationGraph(variable_name = "m",  
                         granularities = [0],
                         aggregation_edges = [(0,0)])


graphs_mobility = [cg_b, cg_c, cg_t, cg_r, cg_od, cg_p, cg_m, ag_b, ag_c, ag_t, ag_r, ag_od, ag_p, ag_m]

# Start data sets:
The following data sets are available:
- ODiN survey: Background information and transportation mode choice for a sample of the Dutch population. (M0|P0, T1)II 
- Administrative: Administrative data on Dutch individuals containing origin, destination, age and income. (B0, OD0|P0, T2)III 
- OSM and OTP: Open street map and OpenTripPlanner route information. (I0|OD0, M0)I 
- Route information: Segments used in routes between origin-destination pairs. (I0|I1, OD0)I 
- Traffic loop counts: Counts of travellers on road segment per minute. (C0|I0, T0)IV

In the report the contexts are defined as follows:
- I: all road segments in the Netherlands
- II: people included in the sample used for the survey
- III: entire Dutch population of people (target population)
- IV: road segments in the Netherlands where traffic loop sensors are located

For the implementation, we take the smallest sets of the above list and use these to create the contexts as sets (which allows for set operations):
- I: roads in sample AND roads excluding sample
- II: people in sample
- III: people in sample AND people including sample
- IV: roads in sample


In [ ]:
# Define the data sets:
# Note: it is not necesarry to name the datasets, but it helps in understanding the use case (because these named data are used in the datasets as well as 
# in the model definitions)

# ODiN Survey
data_odin = Data(left_variables =[Variable("m", 0)],
                 right_variables =[Variable("p", 0),
                                   Variable("t", 1)],
                 context=["people_in_sample"])   # context II
# Administrative
data_admin = Data(left_variables =[Variable("b", 0),
                                   Variable("od", 0)],
                  right_variables =[Variable("p", 0), 
                                    Variable("t", 2)],
                  context=["people_in_sample", "people_excl_sample"])  # context III

# OSM and OTP
data_osm = Data(left_variables =[Variable("r", 0)],
                right_variables =[Variable("od", 0), 
                                  Variable("m", 0)],
                context=["roads_in_sample", "roads_excl_sample"])  # context I
    
# Route information
data_routes = Data(left_variables =[Variable("r", 0)],
                   right_variables =[Variable("od", 0),
                                    Variable("r", 1)],
                   context=["roads_in_sample", "roads_excl_sample"])  # context I
# Traffic loop counts
data_loops = Data(left_variables =[Variable("c", 0)],
                  right_variables =[Variable("r", 0), 
                                    Variable("t", 0)],
                  context=["roads_in_sample"])  # context IV

# start data sets
start_set_mobility = SetOfSources(start_set=[data_odin, data_admin, data_osm, data_routes, data_loops])

# goal
goal_mobility = Data(left_variables =[Variable("c", 0)],
            right_variables =[Variable("r", 0)],
            context=["roads_in_sample", "roads_excl_sample"])  # context I



# Models
The following models are available:
- Modality choice model
- Shortest path model
- Calibration model

In [ ]:
# Modality choice model

class ModelModalityChoice(Model):
    def __init__(self):
        self.name = "Modality Choice model"
        self.input_data = [Data(left_variables =[Variable("b", 0), # source 0
                                                     Variable("od", 0)],
                                    right_variables =[Variable("p", 0),
                                                      Variable("t", 2)],
                                context=["dummyX"]),
                               Data(left_variables =[Variable("m", 0)],  # source 1
                                    right_variables =[Variable("p", 0),
                                                      Variable("t", 2)],
                                    context=["dummyY"])]  
        self.output_data = Data(left_variables =[Variable("p", 0)],
                                    right_variables =[Variable("od", 0),
                                                     Variable("t", 2)],
                                    context=["dummyY"])
        self.context_rule = "custom"
        
        # The modality choice model actually will provide variable m0 and t2, but since they are not 
        # needed in the rest of the use case. We select m0 == "car" and move this now constant variable
        # to the context.
        
            
    def apply(self, potential_input):
        # If each source in the required input (self.input_data) is present in the potential_input, then the model is applicable
        # We "accept a data set" without regarding the context, only the left and right variables must match exactly
        
        # Check if source 0 is availble (variables only)
        source_match_0 = any([ds.equal_nocontext(self.input_data[0]) for ds in potential_input])
        source_match_1 = any([ds.equal_nocontext(self.input_data[1]) for ds in potential_input])
        
        if source_match_0 and source_match_1:
            # For all sources that match based on variables, we take the context so we can apply the context check(s)
            context_matches_0 = [ds.get_context() for ds in potential_input if ds.equal_nocontext(self.input_data[0])]
            context_matches_1 = [ds.get_context() for ds in potential_input if ds.equal_nocontext(self.input_data[1])]
            
            # Output list: here we will add any legal outcomes based on the inputs
            output_list = []

            # We have one or more matches for the both data sources. Now let's check if there is a combination that satisfies the context condition 
            # For this model: C1 is a subset of C0, then the context is C0

            for c0, c1 in itertools.product(context_matches_0, context_matches_1):
                # Check all combinations of potential C0's and C1's
                if c1.issubset(c0): 
                    # Yes, there is a combination that satisfies the condition
                    # context X is a subset of context Y
                    output_data_temp = copy.deepcopy(self.output_data)  # copy the output_data
                    output_data_temp.context = c0  # overwrite the context (output is Y)
                    output_list.append(output_data_temp)
            if len(output_list)>0:
                return output_list
        else:
            return False
            

In [ ]:
# Shortest path model
class ModelShortestPath(Model):
    def __init__(self):
        self.name = "Shortest Path model"
        self.input_data = [Data(left_variables =[Variable("p", 0)], # source 0
                                    right_variables =[Variable("od", 0),
                                                     Variable("t", 2)],  
                                context=["dummyX"]),
                               Data(left_variables =[Variable("r", 0)],  # source 1
                                    right_variables =[Variable("od", 0),
                                                      Variable("r", 1)],
                                    context=["dummyY"])]  
        self.output_data = Data(left_variables =[Variable("p", 0)],
                                    right_variables =[Variable("r", 1),
                                                     Variable("t", 2)],
                                    context=["dummyY"])
        self.context_rule = "custom"
        
            
    def apply(self, potential_input):
        # If each source in the required input (self.input_data) is present in the potential_input, then the model is applicable
        # We "accept a data set" without regarding the context, only the left and right variables must match exactly
        
        # For this model, the output context is equal to the context of source 1 (so the second model in the list: (R0|R1, OD0))
        
        # Check if source 0 is availble (variables only)
        source_match_0 = any([ds.equal_nocontext(self.input_data[0]) for ds in potential_input])
        source_match_1 = any([ds.equal_nocontext(self.input_data[1]) for ds in potential_input])
        
        if source_match_0 and source_match_1:
            # Check if source 1 is available 
            # For all sources that match based on variables, we take the context so we can apply the context check(s)
            context_matches_1 = [ds.get_context() for ds in potential_input if ds.equal_nocontext(self.input_data[1])]

            # Output list: here we will add any legal outcomes based on the inputs
            output_list = []

            # We have one or more matches for data source. Now let's check if there is a combination that satisfies the context condition 
            # For this model: C1 is a subset of C0, then the context is C0

            for c1 in context_matches_1:
                # This context will be the context of the output source
                output_data_temp = copy.deepcopy(self.output_data)  # copy the output_data
                output_data_temp.context = c1  # overwrite the context 
                output_list.append(output_data_temp)
                
            if len(output_list)>0:
                
                return output_list
        else:
            return False
            



In [ ]:
# Calibration model

# Shortest path model
class ModelCalibration(Model):
    def __init__(self):
        self.name = "Calibration model"
        self.input_data = [Data(left_variables =[Variable("c", 0),
                                                Variable("p", 0)], # source 0
                                    right_variables =[Variable("r", 0),
                                                     Variable("t", 2)],
                                context=["dummyX"]),
                               Data(left_variables =[Variable("p", 0)],  # source 1
                                    right_variables =[Variable("r", 0),
                                                     Variable("t", 2)],
                                    context=["dummyY"])]  
        self.output_data = Data(left_variables =[Variable("c", 0)],
                                    right_variables =[Variable("r", 0)],
                                    context=["dummyY"])
        self.context_rule = "custom"
        
        # The variable t_2 can be selected as t_2 == "morning rush hour" and be constant. It is then 
        # put in the context.
            
    def apply(self, potential_input):
        # If each source in the required input (self.input_data) is present in the potential_input, then the model is applicable
        # We "accept a data set" without regarding the context, only the left and right variables must match exactly
        
        # For this model, the output context is equal to the context of source 1 (so the second model in the list: (R0|R1, OD0))
        
        # Check if source 0 is availble (variables only)
        source_match_0 = any([ds.equal_nocontext(self.input_data[0]) for ds in potential_input])
        source_match_1 = any([ds.equal_nocontext(self.input_data[1]) for ds in potential_input])
        
        if source_match_0 and source_match_1:
            # For all sources that match based on variables, we take the context so we can apply the context check(s)
            context_matches_1 = [ds.get_context() for ds in potential_input if ds.equal_nocontext(self.input_data[1])]

            # Output list: here we will add any legal outcomes based on the inputs
            output_list = []

            # We have one or more matches for data source. Now let's check if there is a combination that satisfies the context condition 
            # For this model: C1 is a subset of C0, then the context is C0

            for c1 in context_matches_1:
                # This context will be the context of the output source
                output_data_temp = copy.deepcopy(self.output_data)  # copy the output_data
                output_data_temp.context = c1  # overwrite the context 
                output_list.append(output_data_temp)
            if len(output_list)>0:
                return output_list
        else:
            return False
            


In [ ]:
m1 = ModelModalityChoice()
m2 = ModelShortestPath()
m3 = ModelCalibration()
models_mobility = [m1, m2, m3]

# Test case definition
Create the test object.

In [ ]:
test_mobility = TestCase(goal=goal_mobility, 
                         start_set=start_set_mobility, 
                         graphs=graphs_mobility,
                         models = models_mobility)